CNN using Custom dataset in pytorch framework

In [79]:
import torch
from torchvision.transforms import transforms
from torch.utils.data import random_split,DataLoader

In [2]:
transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))#normalize the data
])

In [3]:
from torchvision import datasets

In [4]:
dataset=datasets.ImageFolder(root="D:\\images",transform=transform)

In [5]:
print(len(dataset))

35


In [6]:
print(dataset.classes)

['car', 'dhoni', 'nature']


In [52]:
print(dataset[0])

(tensor([[[-0.4039, -0.3961, -0.3961,  ..., -0.5059, -0.5059, -0.5137],
         [-0.3961, -0.4039, -0.4118,  ..., -0.4980, -0.4980, -0.5059],
         [-0.3882, -0.4039, -0.4196,  ..., -0.4667, -0.4667, -0.4745],
         ...,
         [ 0.2078,  0.3176,  0.4431,  ..., -0.6627, -0.6157, -0.5765],
         [ 0.5137,  0.4588,  0.3333,  ..., -0.6784, -0.6314, -0.5922],
         [ 0.1059,  0.0667, -0.0118,  ..., -0.6706, -0.6314, -0.5922]],

        [[-0.2627, -0.2549, -0.2549,  ..., -0.3490, -0.3490, -0.3569],
         [-0.2549, -0.2627, -0.2706,  ..., -0.3412, -0.3412, -0.3490],
         [-0.2627, -0.2784, -0.2941,  ..., -0.3255, -0.3255, -0.3333],
         ...,
         [ 0.2078,  0.3176,  0.4431,  ..., -0.7569, -0.7098, -0.6706],
         [ 0.5137,  0.4588,  0.3333,  ..., -0.7725, -0.7255, -0.6863],
         [ 0.1059,  0.0667, -0.0118,  ..., -0.7647, -0.7255, -0.6863]],

        [[-0.0902, -0.0824, -0.0824,  ..., -0.1608, -0.1608, -0.1686],
         [-0.0824, -0.0902, -0.0980,  ..., -

In [82]:
#splitting 80% of data to training
train_size=int(0.8*len(dataset))

In [83]:
train_size

28

In [84]:
#splitting remaining data to the test
test_size=len(dataset)-train_size

In [10]:
test_size

7

In [11]:
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])

In [12]:
train_loader=DataLoader(train_dataset,batch_size=4,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=4,shuffle=True)

In [13]:
#neural network
import torch.nn as nn

In [14]:
class CNNModel(nn.Module):
    def __init__(self,):
        super().__init__()

        #formula =((w-f+2p)/s+)1
        #w-width and height,f-filter,p-padding,s-stride

        #input size :3 x 224 x224
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.relu1=nn.ReLU()
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)

        #input size :12 x 112 x112
        self.conv2=nn.Conv2d(in_channels=12,out_channels=24,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()
        self.pool2=nn.MaxPool2d(kernel_size=2,stride=2)

        #input size :24 x 56 x 56
        self.linear1=nn.Linear(24*56*56,128)
        self.relu3=nn.ReLU()
        self.linear2=nn.Linear(128,3)
    
    def forward(self,data):
        out=self.conv1(data)
        out=self.relu1(out)
        out=self.pool(out)

        out=self.conv2(out)
        out=self.relu2(out)
        out=self.pool(out)

        out=out.view(-1,24*56*56)#flatten layer 

        out=self.linear1(out)
        out=self.relu3(out)
        out=self.linear2(out)#we no need to mention softmax it automatically call it
        return out


In [15]:
model=CNNModel()

In [16]:
import torch.optim as optim

In [17]:
learning_rate=0.01

In [18]:
criterion=nn.CrossEntropyLoss()

In [19]:
optimizer=optim.Adam(model.parameters(),learning_rate)

In [23]:
#training part 
def neuralnetwork(model,criterion,optimizer,epochs=20):
    for epoch in range(epochs):
        model.train()
        for input,labels in train_loader:
            running_loss=0
            output=model(input)
            optimizer.zero_grad()

            loss=criterion(output,labels)

            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
        print(f"the epochs are {epoch+1}/{epochs} and loss is {running_loss/len(train_loader)}")

In [24]:
neuralnetwork(model,criterion,optimizer,15)

the epochs are 1/15 and loss is 8.514940321089983e-08
the epochs are 2/15 and loss is 2.2564576543767805e-07
the epochs are 3/15 and loss is 2.1712790580100512e-06
the epochs are 4/15 and loss is 1.0728674949080284e-06
the epochs are 5/15 and loss is 1.2772401143073304e-07
the epochs are 6/15 and loss is 0.0
the epochs are 7/15 and loss is 1.1920909465905944e-07
the epochs are 8/15 and loss is 1.958392203960102e-06
the epochs are 9/15 and loss is 5.023784654310605e-07
the epochs are 10/15 and loss is 3.405978889402052e-08
the epochs are 11/15 and loss is 5.917858548595437e-07
the epochs are 12/15 and loss is 9.110885912377853e-07
the epochs are 13/15 and loss is 9.281188145645761e-07
the epochs are 14/15 and loss is 1.0217926923620066e-07
the epochs are 15/15 and loss is 4.257474373048353e-09


In [80]:
#testing part
def testingpart(model,test_loader):
    total=0
    correct=0
    with torch.no_grad():
        for inputs,labels in test_loader:
            outputs=model(inputs)
            print(outputs)
            _,predicted=torch.max(outputs.data,1)#it return the max value in the output & also 
                                                 #which class they belong too
            print("max values",_)
            print("class is",predicted)
            total=labels.size(0)
            correct+=(predicted==labels).sum().item()
        print(f"accuracy of the model is {correct/total}")


In [81]:
testingpart(model,test_loader)

tensor([[-45.9515,  72.9120, -24.8831],
        [  5.2437, -17.2137,   7.4001],
        [ 16.1823, -17.5656,   3.7175],
        [  7.9327, -10.4727,   2.2830]])
max values tensor([72.9120,  7.4001, 16.1823,  7.9327])
class is tensor([1, 2, 0, 0])
tensor([[  2.2721,  -2.2164,   0.7290],
        [-56.0631,  78.2264, -23.3306],
        [  1.2630,  -8.7438,   8.4174]])
max values tensor([ 2.2721, 78.2264,  8.4174])
class is tensor([0, 1, 2])
accuracy of the model is 1.0
